# Introduction 

DialoGPT(Dialogue Generative Pre-trained Transformer)는 neural conversational response generation을 다루기 위해 GPT-2를 확장한다.

GPT-2와 같이 DialoGPT는 Auto Regressive(AR) Language Model(LM)이며 multi-layer transformer를 사용한다. 그러나 GPT-2와 다르게 Reddit discussion chain에서 추출된 대규모 대화 pairs/sessions으로 학습된다. 논문에서는 이 대규모 대화 pairs/sessions이 DialoGPT가 대화 흐름에서 $P(Target, Source)$에 대한 joint distribution을 포착할 수 있게 만들었다. 

# Dataset

데이터셋은 2005년부터 2017년에 걸쳐 Reddit에서 스크랩된 comment chain에서 추출된다. 아래 기준에 해당하는 데이터를 필터링한다.



1.   URL이 있는 source나 target
2.   3개 이상의 단어 반복이 target에 포함된 경우
3.   응답이 가장 자주 사용하는 top 50 단어(a, the, of 등)가 적어도 하나 이상 포함되지 않은 경우 
4.   응답에 ["또는"] 이 포함된 경우
5.   source와 target 시퀀스가 합쳐서 200단어보다 긴 경우
6.   target이 offensive language를 포함한 경우
7.   하위 레딧에 많은 수가 offensive한 내용을 포함할 가능성이 많다고 인식되는 경우
8.   단조로운 문장 적극적으로 배제 


필터링 후 데이터 세트는 총 18억 개의 단어로 147,116,725개의 대화 인스턴스로 구성된다. 



# Method

- Model architecture

  - 우리는 첫 번째로 대화세션 안에서 모든 대화 turns을 concat 시켜 긴 text $x_1, ..., x_N$($N$은 시퀀스 길이)를 만들고 끝에는 end-of-text-token을 넣는다. 
  - source sentence(대화 히스토리)는 $S = x_1, ..., x_m$으로 표기하고 target sentence (ground-truth response)는 $T = x_{m+1}, ..., x_N$으로 표기한다.
  - 이때, $P(T|S)$은 조건부 확률의 일련의 곱으로 아래의 식과 같이 쓰여진다. 
  > $p(T|s) = \prod_{n = m+1}^N p(x_n | x_1,..., x_{n-1}) $
  - DialoGPT는 GPT-2를 따라 multi-turn dialogue를 하나의 text로 간주한다.
  - 따라서 multi-turn dialogue session인 $T_1, ..., T_k$은 $p(T_k, ..., T_2|T_1)$로 볼 수 있고 이는 사실 $p(T_i|T_1,...,T_{i-1})$ (여기서 $i$는 $m+1$) 조건부 확률을 product한 것이다. 
  - 결과적으로 $p(T_k, ..., T_2|T_1)$을 최적화하는 것은 모든 $p(T_I | T_1, ..., T_{i-1})$ source-target 페어를 최적화하는 것이다.





# Code

## Mount

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Install & Import library

In [ ]:
!pip install transformers

In [ ]:
!unzip '/content/drive/MyDrive/Dialo-20221218T162952Z-001.zip'

In [ ]:
%cd '/content/Dialo'

In [15]:
!mv config.py medium/
!mv discord_bot.py medium/
!mv interact.py medium/

In [ ]:
%cd '/content/Dialo/medium'

# Run "interact.py"

py파일을 바로 돌려도 된다



In [ ]:
import torch
import torch.nn.functional as F
from transformers import GPT2Tokenizer, GPT2LMHeadModel, GPT2Config
from config import device_f, device_r, num_samples, MMI_temperature, top_k

torch.set_grad_enabled(False)

tokenizer = GPT2Tokenizer('medium/vocab.json', 'medium/merges.txt')

weights = torch.load('medium/medium_ft.pkl')
# fix misused key value
weights["lm_head.weight"] = weights["lm_head.decoder.weight"]
weights.pop("lm_head.decoder.weight", None)

cfg = GPT2Config.from_json_file('medium/config.json')
model: GPT2LMHeadModel = GPT2LMHeadModel(cfg)
model.load_state_dict(weights,strict=False)
if device_f == 'cuda':
    model.half()
model.to(device_f)
model.eval()

weights = torch.load('medium/small_reverse.pkl')
# fix misused key value
weights["lm_head.weight"] = weights["lm_head.decoder.weight"]
weights.pop("lm_head.decoder.weight", None)

reverse_model: GPT2LMHeadModel = GPT2LMHeadModel(cfg)
reverse_model.load_state_dict(weights,strict=False)
if device_r == 'cuda':
    reverse_model.half()
reverse_model.to(device_r)
reverse_model.eval()


end_token = torch.tensor([[50256]], dtype=torch.long)


def _get_response(output_token, past):
    out = torch.tensor([[]], dtype=torch.long, device=device_f)

    while True:
        util = model.forward(output_token, past_key_values=past)
        output_token, past = util['logits'],util['past_key_values']
        output_token = output_token[:, -1, :].float()
        indices_to_remove = output_token < torch.topk(output_token, top_k)[0][..., -1, None]
        output_token[indices_to_remove] = -float('Inf')
        output_token = torch.multinomial(F.softmax(output_token, dim=-1), num_samples=1)

        out = torch.cat((out, output_token), dim=1)

        if output_token.item() == end_token.item():
            break

    return out, past


def _score_response(output_token, correct_token):
    inputs = torch.cat((output_token, correct_token), dim=1)
    mask = torch.full_like(output_token, -100, dtype=torch.long)
    labels = torch.cat((mask, correct_token), dim=1)

    score = -reverse_model(inputs, labels=labels)['loss'].float()

    return score


def append_messages(old_list: list, new_list: list, truncate_length=64):
    for message in new_list:
        if message != '':
            input_token = tokenizer.encode(message, return_tensors='pt')
            input_token = torch.cat((input_token, end_token), dim=1)
            old_list.append(input_token)

    if len(old_list) == 0:
        old_list.append(end_token)

    # truncate
    total_length = 0
    for i, message in enumerate(reversed(old_list)):
        total_length += message.shape[1]
        if total_length > truncate_length:
            old_list[:] = old_list[-i:]


def generate_message(message_list: list, focus_last_message=True):
    total_input = torch.cat(message_list, dim=1).to(device_f)
    if focus_last_message:
        total_input_reversed = message_list[-1]
    else:
        total_input_reversed = torch.cat(list(reversed(message_list)), dim=1)

    past = None
    if total_input.shape[1] > 1:
        past = model(total_input[:, :-1])

    results = []
    for i in range(num_samples):
        result = _get_response(total_input[:, -1:], past['past_key_values'])
        score = _score_response(result[0].to(device_r), total_input_reversed.to(device_r))
        results.append(result + (score,))

    scores = torch.stack([x[2] for x in results], dim=0)
    winner = torch.multinomial(F.softmax(scores / MMI_temperature, dim=0), num_samples=1).item()
    # winner = torch.argmax(scores, dim=0)

    out = results[winner][0]

    return tokenizer.decode(out.tolist()[0], skip_special_tokens=True)


my_message_list = []
while True:
    print("usr >> ",end="")
    my_message = input()
    if my_message=="quit":
      print("bot >> Quit. Chating End")
      break
    append_messages(my_message_list, [my_message])
    my_response = generate_message(my_message_list)
    print('bot >>', my_response)

    append_messages(my_message_list, [my_response])